# Set up

## Check configuration
Should return path to correct python version (from virtual environment)

In [17]:
import sys
print(sys.executable)
# print('\n'.join(sys.path[:6]))

c:\Users\WB499754\wb-projects\my_packages\tidysdmx\.venv\Scripts\python.exe


## Load libraries

In [1]:
# Automatically reload modules before execution of each cell
# so when you edit src/mypackage/*.py in your editor and rerun cells, 
# changes appear immediately.
%reload_ext autoreload
%autoreload 2

# python
from __future__ import annotations

# Standard library
from pathlib import Path

# Third-party
from pysdmx.model import FixedValueMap, ImplicitComponentMap, ValueMap, MultiValueMap, ComponentMap
from openpyxl import Workbook, load_workbook
import pandas as pd
import pysdmx as px
import pickle as pkl
from datetime import datetime


# Custom
## Functions
from tidysdmx import (
    filter_tidy_raw, 
    validate_dataset_local, 
    map_structures, 
    infer_schema, 
    infer_role_dimension, 
    apply_fixed_value_maps, 
    apply_implicit_component_maps, 
    build_date_pattern_map,
    build_value_map_list,
    build_multi_value_map_list,
    build_representation_map,
    build_single_component_map,
    extract_component_ids,
    write_excel_mapping_template,
    build_structure_map,
    create_schema_from_table,
    build_structure_map_from_template_wb
)


## Define globals

In [14]:
# CAUTION! FOR TESTING ONLY. DO NOT USE IN PRODUCTION.
# os.environ["PYTHONHTTPSVERIFY"] = "0"

# FMR and artefacts information
fmr_url = "https://fmrqa.worldbank.org/FMR/sdmx/v2"
# raw schema
# raw_structure_agency = "WB"
# raw_structure_id = "IFPRI_ASTI"
# raw_structure_version = "1.0"
# dissemination schema
dis_structure_agency = "WB.GGH.HSP"
dis_structure_id = "DS_ASPIRE"
dis_structure_version = "1.0.0"
# structure map
# raw_structure_map = "SM_IFPRI_ASTI_TO_DATA360"

# Path to raw data
path_to_raw_data = Path(
    "../TMP/data/WB_ASPIRE"
)
path_to_xlsx_mapping = Path(
    "./data/WB_ASPIRE_MODIFIED_COPY.xlsx"
)

## Initiate API client

In [3]:
print(fmr_url)
client = px.api.fmr.RegistryClient(fmr_url)
client

https://fmrqa.worldbank.org/FMR/sdmx/v2


# STEP 1 - Load raw data

Here we are loading the raw dataset as provided from the source. In this demonstration notebook, the raw data is simply being loaded from file, but in the final pipeline, the provenance of the file should be fully documented in a configuration file, and read from the source / DDH possible.

In [5]:
def read_raw_data(folder_path):
    """Read all .csv files from a folder and return a single pandas DataFrame.

    Parameters
    ----------
    folder_path : str | pathlib.Path
        Path to the folder containing CSV files.

    Returns:
    -------
    pandas.DataFrame
        Concatenated DataFrame of all CSVs (same structure assumed), with a
        "Series code" column indicating the source filename (without .csv).
    """
    folder = Path(folder_path)
    csv_files = sorted(folder.glob("*.csv"))
    if not csv_files:
        raise ValueError(f"No CSV files found in folder: {folder}")

    dfs = []
    for f in csv_files:
        df = pd.read_csv(f)
        df["Series code"] = f.stem  # filename without extension
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

raw_df = read_raw_data(path_to_raw_data)
raw_df.head()

,economy,YR1998,YR1999,YR2000,YR2001,YR2002,YR2003,YR2004,YR2005,YR2006,...,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,Series code
0,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,per_allsp.adq_ep_preT_tot
1,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,166.653401,NaN,NaN,NaN,per_allsp.adq_ep_preT_tot
2,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,423.624967,350.713358,408.467464,380.697320,NaN,NaN,NaN,per_allsp.adq_ep_preT_tot
3,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.347211,...,76.066882,112.592261,NaN,74.301746,70.919223,61.428587,58.905471,70.119141,62.90742,per_allsp.adq_ep_preT_tot
4,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,102.066885,112.068938,112.806282,123.925970,123.501921,120.529084,116.753183,NaN,NaN,per_allsp.adq_ep_preT_tot


# STEP 2: Reshape raw data

A critical step of this opinionated pipeline framework is to systematically reshape raw into tidy format (one observation per row). For more information about tidy data, please refer to [Hadley Wickam's original paper](https://vita.had.co.nz/papers/tidy-data.pdf). 

This step is critical because once data has been reshaped into a tidy format, the rest of the pipeline can be fully standradized, bringing immediate maintenance, scalability, and insititutional knowledge benefits. 

This is also a good place to implement minimal data cleaning if necessary.

In [8]:
def reshape_raw_data(df: pd.DataFrame) -> pd.DataFrame:
    """Reshape raw data and implements basic data cleaning.

    It 'melts' (unpivots) columns starting with 'data.' into two columns ('name' and 'value'),
    and then cleans the 'name' column by removing the 'data.' prefix.

    Args:
        df: The input pandas DataFrame containing columns like 'data.1', 'data.2', etc.

    Returns:
        A new DataFrame in the longer format.
    """
    # 1. Equivalent of R's pivot_longer (using melt)
    # Selects columns starting with 'data.' for unpivoting
    data_cols = df.filter(like='YR').columns.tolist()

    df_lg = df.melt(
        id_vars=[col for col in df.columns if col not in data_cols], # Keep all non-data columns as identifier variables
        var_name='year',    # New column for the original column names
        value_name='value', # New column for the values
    )

    # 2. Equivalent of R's stringr::str_replace
    # Removes the 'data.' prefix from the 'name' column
    df_lg['year'] = df_lg['year'].str.replace('YR', '', regex=False)

    # Filter out rows with missing values in 'value'
    df_lg = df_lg.dropna(subset=['value'])
    
    return df_lg

tidy_raw_df = reshape_raw_data(raw_df)
tidy_raw_df.head()

,economy,Series code,year,value
42,GMB,per_allsp.adq_ep_preT_tot,1998,6.120197
182,GMB,per_allsp.adq_ep_tot,1998,5.098720
322,GMB,per_allsp.adq_pop_preT_tot,1998,8.272170
602,GMB,per_allsp.adq_pop_tot,1998,8.272170
742,GMB,per_allsp.adq_pop_urb,1998,9.228071


# STEP 3: Describe the tidy raw data input

We will describe the tidy raw data input using an SDMX schema. This description will allow for early validation of our input data during subsequent run of the pipelines for data updates. 

The `create_schema_from_table()` helper function allows pipeline developers to create pysdmx schema object automatically with minimal inputs from the pipeline developers.

In [9]:
tidy_raw_schema=create_schema_from_table(
    tidy_raw_df, 
    dimensions=["Series code", "economy"],
    time_dimension="year", 
    measure="value")

tidy_raw_schema.components["Series code"]

Component(id='Series code', required=True, role=Role.DIMENSION, concept=Concept(id='Series code', name='Series code', description='Concept inferred from column Series code', dtype=DataType.STRING), local_dtype=DataType.STRING, name='Series code', description='Concept inferred from column Series code', local_codes=Codelist(id='CL_SERIES_CODE', name='Codelist for Series code', version='1.0', agency='SDMX', items=[Code(id='PER_ALLSP_ADQ_EP_PRET_TOT', name='per_allsp.adq_ep_preT_tot'), Code(id='PER_ALLSP_ADQ_EP_TOT', name='per_allsp.adq_ep_tot'), Code(id='PER_ALLSP_ADQ_POP_PRET_TOT', name='per_allsp.adq_pop_preT_tot'), Code(id='PER_ALLSP_ADQ_POP_RUR', name='per_allsp.adq_pop_rur'), Code(id='PER_ALLSP_ADQ_POP_TOT', name='per_allsp.adq_pop_tot'), Code(id='PER_ALLSP_ADQ_POP_URB', name='per_allsp.adq_pop_urb'), Code(id='PER_ALLSP_ADQ_Q1_PRET_TOT', name='per_allsp.adq_q1_preT_tot'), Code(id='PER_ALLSP_ADQ_Q1_RUR', name='per_allsp.adq_q1_rur'), Code(id='PER_ALLSP_ADQ_Q1_TOT', name='per_allsp.adq

# STEP 4: Filter out unnecessary rows (Optional)

In [10]:
def apply_constraints(df: pd.DataFrame, constraints: Dict[str, List]) -> pd.DataFrame:
    """Filters a DataFrame based on a dictionary of column names and valid values.
    
    Args:
        df (pd.DataFrame): The source dataframe.
        constraints (dict): A dict where keys are column names and values are 
                        lists of valid entries to keep (e.g., {'col': ['val1', 'val2']}).
                        
    Returns:
        pd.DataFrame: A filtered copy of the original dataframe.
    """
    for column, valid_values in constraints.items():
        # strict check: ensure column exists to avoid KeyErrors
        if column in df.columns:
            df = df[df[column].isin(valid_values)]
        else:
            print(f"Warning: Column '{column}' not in DataFrame. Skipping.")
            
    return df


constraints = {
    "Series code": ["per_allsp.adq_ep_preT_tot", "per_allsp.adq_ep_tot", "per_allsp.adq_pop_preT_tot"]#,
    # "TIME_PERIOD": ["1992"],
    # "AREA": ["GHA"]
}


tidy_raw_df=apply_constraints(tidy_raw_df, constraints)

# STEP 5: Create structure map

## Fetch dissemination schema

In [11]:
dis_schema = client.get_schema("datastructure", agency=dis_structure_agency, id=dis_structure_id, version=dis_structure_version)
dis_schema

Schema(context='datastructure', agency='WB.GGH.HSP', id='DS_ASPIRE', components=Components(data=[Component(id='REF_AREA', required=True, role=Role.DIMENSION, concept=Concept(id='AREA', urn='urn:sdmx:org.sdmx.infomodel.conceptscheme.Concept=WB:CS_WB(1.0.0).AREA', name='Area', dtype=DataType.STRING, codes=Codelist(id='CL_AREA', name='World Bank Reference Area Code List', version='1.0', agency='WB', items=[Code(id='NZL', name='New Zealand', description='New Zealand'), Code(id='FJI', name='Fiji', description='Fiji'), Code(id='PNG', name='Papua New Guinea', description='Papua New Guinea'), Code(id='GLP', name='Guadeloupe', description='Guadeloupe'), Code(id='STP', name='Sao Tome and Principe', description='Sao Tome and Principe'), Code(id='MHL', name='Marshall Islands', description='Marshall Islands'), Code(id='WLF', name='Wallis-et-Futuna (Fr.)', description='Wallis and Futuna'), Code(id='CUB', name='Cuba', description='Cuba'), Code(id='SDN', name='Sudan', description='Sudan'), Code(id='GM

## Create structure map from mapping template

In [15]:
mappings = pd.read_excel(path_to_xlsx_mapping, sheet_name=None, engine="openpyxl")
sm=build_structure_map_from_template_wb(mappings)

# Map data to dissemination schema

## Load edited mapping template as pysdmx schema

In [58]:
wb=load_workbook(path_to_xlsx_mapping)
sm=build_structure_map(wb)
sm.maps

[FixedValueMap(target='FREQ', value='A'),
 ImplicitComponentMap(source='AREA', target='REF_AREA'),
 ComponentMap(source='INDICATOR', target='INDICATOR', values=RepresentationMap(id='REPMAP_INDICATOR', name='Mapping for INDICATOR', version='1.0', agency='DEFAULT_AGENCY', maps=[ValueMap(source='RES.FEMALE.TOT.FTE', target='RES_FEMALE_TOT_FTE'), ValueMap(source='RES.MALE.TOT.FTE', target='RES_MALE_TOT_FTE'), ValueMap(source='RES.TOT.FTE', target='RES_TOT_FTE')])),
 ComponentMap(source='INDICATOR', target='SEX', values=RepresentationMap(id='REPMAP_SEX', name='Mapping for SEX', version='1.0', agency='DEFAULT_AGENCY', maps=[ValueMap(source='RES_FEMALE_TOT_FTE', target='F'), ValueMap(source='RES_MALE_TOT_FTE', target='M'), ValueMap(source='RES_TOT_FTE', target='_T')])),
 FixedValueMap(target='AGE', value='_T'),
 ComponentMap(source='AREA', target='URBANISATION', values=RepresentationMap(id='REPMAP_URBANISATION', name='Mapping for URBANISATION', version='1.0', agency='DEFAULT_AGENCY', maps=[Va

## Implement mapping

In [59]:
out = map_structures(df = tidy_raw_df, structure_map = sm)

# STEP 6: Final validation

In [60]:
dis_errors = validate_dataset_local(df = out, schema = dis_schema, sdmx_cols=[])
dis_errors

,Validation,Error
0,columns,Found unexpected column: NOTE
1,codelist_ids,Invalid values found in column 'INDICATOR': ['...
2,missing_values,Missing values found in mandatory columns:\n ...
